# Arcee AI Virtuoso Large Model Demo via Together.ai

This notebook demonstrates how to use the [Arcee AI Virtuoso Large model](https://api.together.ai/models/arcee-ai/virtuoso-large) through the Together.ai API.


## Install Required Packages


In [ ]:
!pip install -U pip
!pip install -qU together
print("✅ Required packages installed")

## Set Up API Key

We'll use the API key stored in the environment variable `TOGETHER_API_KEY`.


In [ ]:
import os
import together
from IPython.display import Markdown, display, clear_output

# Initialize the Together client
api_key = os.environ.get("TOGETHER_API_KEY")

# Verify API key is available
if api_key is None:
    print("⚠️ TOGETHER_API_KEY environment variable not found. Please set it before proceeding.")
    print("You can set it with: import os; os.environ['TOGETHER_API_KEY'] = 'your_api_key_here'")
else:
    # Create a client instance
    client = together.Together(api_key=api_key)
    print("✅ API key loaded successfully and client initialized")


## Basic Text Completion


In [ ]:
def generate_text(prompt, max_tokens=256):
    """Generate text using the Arcee AI Virtuoso Large model"""
    try:            
        response = client.completions.create(
            prompt=prompt,
            model="arcee-ai/virtuoso-large",
            max_tokens=max_tokens,
            temperature=0.7,
        )
        
        # Access the response as an object, not a dictionary
        return response.choices[0].text
    except Exception as e:
        return f"Error generating text: {str(e)}"

# Example prompt
prompt = "Explain the concept of transfer learning in machine learning in simple terms:"

print(f"Generating text for prompt: '{prompt}'")
response = generate_text(prompt)
display(Markdown(f"**Prompt:** {prompt}\n\n**Response:**\n{response}"))


## Chat Completion


In [ ]:
def chat_completion(messages, max_tokens=256):
    """Generate a chat completion using the Arcee AI Virtuoso Large model"""
    try:        
        # Try to use the chat completions API first
        try:
            response = client.chat.completions.create(
                model="arcee-ai/virtuoso-large",
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.7,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Chat completions error: {e}")
            print("Falling back to completions API with manually formatted prompt...")
            
            # Manually format the chat messages into a prompt as fallback
            formatted_prompt = format_chat_prompt(messages)
            
            # Use the completions API
            response = client.completions.create(
                prompt=formatted_prompt,
                model="arcee-ai/virtuoso-large",
                max_tokens=max_tokens,
                temperature=0.7,
            )
            
            # Return the generated text
            return response.choices[0].text
    except Exception as e:
        return f"Error in chat completion: {str(e)}"

def format_chat_prompt(messages):
    """Manually format chat messages into a prompt string"""
    prompt = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        
        if role == "system":
            prompt += f"System: {content}\n\n"
        elif role == "user":
            prompt += f"User: {content}\n\n"
        elif role == "assistant":
            prompt += f"Assistant: {content}\n\n"
    
    prompt += "Assistant: "
    return prompt

# Example chat
messages = [
    {"role": "system", "content": "You are a helpful AI assistant that provides concise and accurate information."},
    {"role": "user", "content": "What are the key differences between supervised and unsupervised learning?"}
]

print("Generating chat completion...")
response = chat_completion(messages)
display(Markdown(f"**System:** {messages[0]['content']}\n\n**User:** {messages[1]['content']}\n\n**Assistant:**\n{response}"))


## Streaming Response


In [ ]:
def stream_chat(messages, max_tokens=512):
    """Stream a chat completion using the Arcee AI Virtuoso Large model"""
    response = ""
    
    try:
        # Try to use the chat completions streaming API first
        try:
            print("Starting streaming response...")
            stream = client.chat.completions.create(
                model="arcee-ai/virtuoso-large",
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.7,
                stream=True,
            )
            
            for chunk in stream:
                if chunk.choices and chunk.choices[0].delta.content:
                    text = chunk.choices[0].delta.content
                    # Check for contaminated content
                    if "Human:" in text or "User:" in text:
                        print("⚠️ Detected contaminated content in response, truncating...")
                        break
                    response += text
                    clear_output(wait=True)
                    display(Markdown(response))
                    
        except Exception as e:
            print(f"Chat completions streaming error: {e}")
            print("Falling back to non-streaming chat completion...")
            
            # Use regular chat completion instead
            response = chat_completion(messages, max_tokens=max_tokens)
            clear_output(wait=True)
            display(Markdown(response))
            return response
        
        print("\n\nStreaming complete.")
        return response
        
    except Exception as e:
        print(f"Error in stream_chat: {str(e)}")
        return f"Error: {str(e)}"

# Example streaming chat
messages = [
    {"role": "system", "content": "You are a helpful AI assistant that provides concise and accurate information. Keep your responses focused on the question asked."},
    {"role": "user", "content": "Explain the concept of gradient descent in machine learning in 3-4 paragraphs."}
]

print("Starting streaming chat...")
result = stream_chat(messages)
print("\nResponse length:", len(result) if result else 0, "characters")


## Advanced Prompting with System Instructions


In [ ]:
messages = [
    {"role": "system", "content": "You are an expert data scientist who specializes in explaining complex concepts in simple terms. Format your responses using markdown with headers, bullet points, and code examples where appropriate."},
    {"role": "user", "content": "Explain how to implement k-means clustering in Python and provide a simple example."}
]

print("Generating advanced response with system instructions...")
response = chat_completion(messages, max_tokens=512)
print("Response received!")
display(Markdown(response))


## Multi-turn Conversation


In [ ]:
conversation = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is a neural network?"},
    {"role": "assistant", "content": "A neural network is a computational model inspired by the human brain. It consists of interconnected nodes (neurons) organized in layers that process information. Neural networks learn from data by adjusting the connections (weights) between neurons through a process called training. They're particularly effective for tasks like pattern recognition, classification, and prediction."},
    {"role": "user", "content": "What are some common activation functions used in neural networks?"}
]

print("Demonstrating multi-turn conversation...")
print("Previous context: Asked about neural networks")
print("Current question: Asking about activation functions")
response = chat_completion(conversation)
display(Markdown(f"**User:** {conversation[-1]['content']}\n\n**Assistant:**\n{response}"))


## Direct API Call (Fallback Method)


In [ ]:
import requests
import json

def direct_api_call(prompt, max_tokens=256):
    """Make a direct API call to Together.ai for the Arcee AI Virtuoso Large model"""
    try:
        api_key = os.environ.get('TOGETHER_API_KEY')
        if api_key is None:
            return "API key not found. Please set TOGETHER_API_KEY environment variable."
            
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "model": "arcee-ai/virtuoso-large",
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": 0.7
        }
        
        print("Making direct API call to Together.ai...")
        response = requests.post(
            "https://api.together.xyz/v1/completions",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            print("API call successful!")
            return result["choices"][0]["text"]
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error in direct API call: {str(e)}"

# Example direct API call
direct_prompt = "What is the difference between supervised and unsupervised learning?"
print(f"Sending prompt: '{direct_prompt}'")
direct_response = direct_api_call(direct_prompt)
display(Markdown(f"**Prompt:** {direct_prompt}\n\n**Response:**\n{direct_response}"))


## Conclusion

This notebook demonstrated how to use the Arcee AI Virtuoso Large model through the Together.ai API.
